You are given an integer array heights representing the heights of buildings, some bricks, and some ladders.

You start your journey from building 0 and move to the next building by possibly using bricks or ladders.

While moving from building i to building i+1 (0-indexed),

If the current building's height is greater than or equal to the next building's height, you do not need a ladder or bricks.
If the current building's height is less than the next building's height, you can either use one ladder or (h[i+1] - h[i]) bricks.
Return the furthest building index (0-indexed) you can reach if you use the given ladders and bricks optimally.

 

Example 1:


Input: heights = [4,2,7,6,9,14,12], bricks = 5, ladders = 1
Output: 4
Explanation: Starting at building 0, you can follow these steps:
- Go to building 1 without using ladders nor bricks since 4 >= 2.
- Go to building 2 using 5 bricks. You must use either bricks or ladders because 2 < 7.
- Go to building 3 without using ladders nor bricks since 7 >= 6.
- Go to building 4 using your only ladder. You must use either bricks or ladders because 6 < 9.
It is impossible to go beyond building 4 because you do not have any more bricks or ladders.
Example 2:

Input: heights = [4,12,2,7,3,18,20,3,19], bricks = 10, ladders = 2
Output: 7
Example 3:

Input: heights = [14,3,19,3], bricks = 17, ladders = 0
Output: 3
 

Constraints:

1 <= heights.length <= 105
1 <= heights[i] <= 106
0 <= bricks <= 109
0 <= ladders <= heights.length

In [ ]:
class Solution:
    def furthestBuilding(self, heights: list[int], bricks: int, ladders: int) -> int:
        return self.dfs_rec(heights, 0, bricks, ladders)

    def dfs_rec(self, heights, ind, bricks, ladders):
        if ind == len(heights) -1:
            return
        
        # if the cureent hieght is bigger than the next just go.
        if heights[ind] > heights[ind+1]:
            return self.dfs_rec(heights, ind+1, bricks, ladders)
        else:
            # you need to use ladder and bricks and take the maximum building you can reach by the both.
            bricks_side = ind
            ladder_side = ind
            if bricks >= (heights[ind+1] - heights[ind]):
                bricks_side = self.dfs_rec(heights, ind+1, bricks - (heights[ind+1] - heights[ind]), ladders)
            
            if ladders > 0:
                ladder_side = self.dfs_rec(heights, ind+1, bricks, ladders - 1)
        
            return max(bricks_side, ladder_side)


# tc - O(2 ^ n)
# sc - O(n)

In [3]:
Solution().furthestBuilding([4,2,7,6,9,14,12], 5,1)

4

In [4]:
Solution().furthestBuilding(heights = [4,12,2,7,3,18,20,3,19], bricks = 10, ladders = 2)

7

# approach 2:
- you have to choose either of the tools - without trying recussion.
- 👉 “Among all the jumps we’ve taken so far, the biggest ones should use the ladders.”
- so record all the jumps, order it and use the ladder for the bigger ones and use the bricks for the smaller... how ?
-  which means I can keep the using ladder jumps in the heap itself,
- when I am out of ladders, I can use the bricks for the smaller jumps I did so far. - I can find this using min-heap.
- when I get smaller jumps, use min-heap to pop it and use it.

In [ ]:
import heapq

class Solution:
    def furthestBuilding(self, heights: list[int], bricks: int, ladders: int) -> int:
        min_heap = []

        for i in range(len(heights) - 1):
            diff = heights[i + 1] - heights[i]

            if diff > 0:
                heapq.heappush(min_heap, diff)

            # If we used more ladders than we have, we must replace the smallest climb with bricks
            if len(min_heap) > ladders:
                smallest_climb = heapq.heappop(min_heap)
                bricks -= smallest_climb

            # If we run out of bricks, we can't go further
            if bricks < 0:
                # means going to the next building I have no tools, so dont go.
                return i

        return len(heights) - 1


# tc:
# going to all the elements - O(n)
# maintaining heap - O(log n)
# tc - O(n log n)
# sc - O(n)

In [8]:
heights = [4,2,7,6,9,14,12]
bricks = 5
ladders = 1

sol = Solution()
print(sol.furthestBuilding(heights, bricks, ladders))  # Output: 4


4


In [9]:
Solution().furthestBuilding(heights = [4,12,2,7,3,18,20,3,19], bricks = 10, ladders = 2)

7

heights = [4,12,2,7,3,18,20,3,19], bricks = 10, ladders = 2


| **i** | **heights\[i] → heights\[i+1]** | **climb** | **Heap After Push** | **Action**                               | **Bricks Left** | **Ladders Used** |
| ----: | ------------------------------- | --------- | ------------------- | ---------------------------------------- | --------------: | ---------------- |
|     0 | 4 → 12                          | 8         | \[8]                | Heap size ≤ ladders → keep for ladder    |              10 | 0                |
|     1 | 12 → 2                          | -10       | \[8]                | No climb                                 |              10 | 0                |
|     2 | 2 → 7                           | 5         | \[5, 8]             | Heap size ≤ ladders → keep for ladder    |              10 | 0                |
|     3 | 7 → 3                           | -4        | \[5, 8]             | No climb                                 |              10 | 0                |
|     4 | 3 → 18                          | 15        | \[5, 8, 15]         | Heap size > ladders → pop 5 → use bricks |      10 − 5 = 5 | 0                |
|     5 | 18 → 20                         | 2         | \[2, 15, 8]         | Heap size > ladders → pop 2 → use bricks |       5 − 2 = 3 | 0                |
|     6 | 20 → 3                          | -17       | \[8, 15]            | No climb                                 |               3 | 0                |
|     7 | 3 → 19                          | 16        | \[8, 15, 16]        | Heap size > ladders → pop 8 → use bricks |  3 − 8 = **-5** | 0                |
